In [1]:
import sys
import h5py
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks

Using TensorFlow backend.


In [2]:
train_data_path = r'data/american_data/train'
validation_data_path = r'data/american_data/test'

In [3]:
epochs = 50
img_width, img_height = 64, 64
batch_size = 40
samples_per_epoch = 100
validation_steps = 50
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 24
lr = 0.004

In [4]:
model = Sequential()
model.add(Convolution2D(nb_filters1, (conv1_size, conv1_size), padding ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [5]:
model.add(Convolution2D(nb_filters2, (conv2_size, conv2_size), padding ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

C:\Users\anoop\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        8256      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
__________

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 4015 images belonging to 24 classes.
Found 957 images belonging to 24 classes.


In [11]:
#Tensorboard log
#log_dir = './tf-log/'
#tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#cbks = [tb_cb]

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 50,
    #samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    #callbacks=cbks,
    validation_steps=validation_steps)

Epoch 1/50
50/50 [==============================] - 93s - loss: 5.8880 - acc: 0.0440 - val_loss: 3.2942 - val_acc: 0.0722
Epoch 2/50
50/50 [==============================] - 62s - loss: 2.9876 - acc: 0.1515 - val_loss: 2.5609 - val_acc: 0.2407
Epoch 3/50
50/50 [==============================] - 49s - loss: 2.1430 - acc: 0.3549 - val_loss: 2.1444 - val_acc: 0.3616
Epoch 4/50
50/50 [==============================] - 47s - loss: 1.6956 - acc: 0.4685 - val_loss: 1.6853 - val_acc: 0.4273
Epoch 5/50
50/50 [==============================] - 59s - loss: 1.3898 - acc: 0.5480 - val_loss: 1.4246 - val_acc: 0.5251
Epoch 6/50
50/50 [==============================] - 57s - loss: 1.1471 - acc: 0.6330 - val_loss: 1.1056 - val_acc: 0.6289
Epoch 7/50
50/50 [==============================] - 60s - loss: 0.9526 - acc: 0.6807 - val_loss: 1.1248 - val_acc: 0.6640
Epoch 8/50
50/50 [==============================] - 65s - loss: 0.8009 - acc: 0.7425 - val_loss: 1.2987 - val_acc: 0.5878
Epoch 9/50
50/50 [======

In [13]:
target_dir = r'models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save(r'models/model.h5')
model.save_weights('./models/weights.h5')